In [1]:
import tensorflow as tf
import numpy as np
import math

SEPCTURAL_SAMPLES = 10
FEATURE_DIM = SEPCTURAL_SAMPLES*6*2
CONV_LEN = 3
CONV_LEN_INTE = 3#4
CONV_LEN_LAST = 3#5
CONV_NUM = 64
CONV_MERGE_LEN = 8
CONV_MERGE_LEN2 = 6
CONV_MERGE_LEN3 = 4
CONV_NUM2 = 64
INTER_DIM = 128
OUT_DIM = 6#len(idDict)
WIDE = 20
CONV_KEEP_PROB = 0.2

BATCH_SIZE = 64
TOTAL_ITER_NUM = 1000

select = 'a'

metaDict = {'a':[119080, 1193], 'b':[116870, 1413], 'c':[116020, 1477]}
TRAIN_SIZE = metaDict[select][0]
EVAL_DATA_SIZE = metaDict[select][1]
EVAL_ITER_NUM = int(math.ceil(EVAL_DATA_SIZE / BATCH_SIZE))

In [3]:
###### Import training data
train_x = np.load('DeepSense_data/train_x.npy')
train_y = np.load('DeepSense_data/train_y.npy')
eval_x = np.load('DeepSense_data/eval_x.npy')
eval_y = np.load('DeepSense_data/eval_y.npy')

# train_x = np.array(train_x)
print(train_x.shape,train_y.shape)
print(eval_x.shape,eval_y.shape)
# train_x = np.expand_dims(train_x, axis=3)
# acc,gyo = np.split(train_x,2,axis=2)
# acc = acc.reshape((-1,20,60,1))
# gyo = gyo.reshape((-1,20,60,1))
# print(acc.shape,gyo.shape)
print(train_y[0])

(119080, 20, 120) (119080, 6)
(1193, 20, 120) (1193, 6)
[0. 0. 0. 0. 1. 0.]


In [6]:
###### Import training data
train_x = np.load('fre_data.npy')
print(train_x.shape)
train_y = np.load('fre_data_label.npy')
# eval_x = np.load('DeepSense_data/eval_x.npy')
# eval_y = np.load('DeepSense_data/eval_y.npy')
train_x = train_x.reshape((-1,20,120))

# train_x = np.array(train_x)
print(train_x.shape,train_y.shape)
# print(eval_x.shape,eval_y.shape)

acc,gyo = np.split(train_x,2)
acc = acc.reshape((-1,20,60,1))
gyo = gyo.reshape((-1,20,60,1))
print(acc.shape,gyo.shape)
print(train_y[0])

(254000, 120)
(12700, 20, 120) (12700, 6)
(12700, 20, 60, 1) (12700, 20, 60, 1)
[0 0 0 0 1 0]


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [3]:
# Input Data 

training_data_count = len(train_x)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(eval_x)  # 2947 testing series
n_steps = len(train_x[0])  # 128 timesteps per series
n_input = len(train_x[0][0])  # 9 input parameters per timestep


# train_x = (train_x - np.min(train_x))/np.mean(train_x)
print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(train_x.shape, train_y.shape, np.mean(train_x), np.std(train_x))

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(119080, 20, 120, 1) (119080, 6) 0.44252616 9.060849


In [4]:
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Bidirectional
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.core import Activation
from keras.layers import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv3D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras import optimizers
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Reshape
import keras.backend as K
from keras.layers.recurrent import GRU,LSTM
from keras.layers import GRUCell,Lambda
from keras.layers import RNN
from keras import regularizers

Using TensorFlow backend.


In [5]:
n_hidden = 64 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.001
lambda_loss_amount = 0.001
#training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 64
display_iter = 30000  # To show test set accuracy during training


sensor_input = Input(shape=(20,120,1), name='input')

acc_conv1 = Conv2D(CONV_NUM, kernel_size=[1, 2*3*CONV_LEN],
        strides=(1, 2*3), padding='VALID', activation=None, data_format='channels_last')(sensor_input)
acc_conv1 = BatchNormalization()(acc_conv1)
acc_conv1 = Activation('relu')(acc_conv1)
acc_conv1_shape = acc_conv1.get_shape().as_list()
acc_conv1 = Dropout(CONV_KEEP_PROB, noise_shape=[acc_conv1_shape[0], 1, 1, acc_conv1_shape[3]], seed=None)(acc_conv1)
#         layers.dropout(acc_conv1, CONV_KEEP_PROB, is_training=train,
#             noise_shape=[acc_conv1_shape[0], 1, 1, acc_conv1_shape[3]], scope='acc_dropout1')

acc_conv2 = Conv2D(CONV_NUM, kernel_size=[1, CONV_LEN_INTE],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_last')(acc_conv1)
acc_conv2 = BatchNormalization()(acc_conv2)
acc_conv2 = Activation('relu')(acc_conv2)
acc_conv2_shape = acc_conv2.get_shape().as_list()
acc_conv2 = Dropout(CONV_KEEP_PROB, noise_shape=[acc_conv2_shape[0], 1, 1, acc_conv2_shape[3]], seed=None)(acc_conv2)
#layers.dropout(acc_conv2, CONV_KEEP_PROB, is_training=train,
#             noise_shape=[acc_conv2_shape[0], 1, 1, acc_conv2_shape[3]], scope='acc_dropout2')

acc_conv3 = Conv2D(CONV_NUM, kernel_size=[1, CONV_LEN_LAST],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_last')(acc_conv2)
acc_conv3 = BatchNormalization()(acc_conv3)
acc_conv3 = Activation('relu')(acc_conv3)
acc_conv3_shape = acc_conv3.get_shape().as_list()
print('acc_conv3_shape',acc_conv3_shape)
outputs = Reshape((acc_conv3_shape[1], -1))(acc_conv3)

outputs = Bidirectional(LSTM(n_hidden,unit_forget_bias=True,return_sequences=True))(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Bidirectional(LSTM(n_hidden,unit_forget_bias=True,return_sequences=True))(outputs)
outputs = Dropout(0.5)(outputs)

# def reduce_dim(x):
#     return x[:,-1:,:]

# outputs = Lambda(reduce_dim)(outputs)
outputs = Activation('linear')(outputs)
outputs = Flatten()(outputs)

outputs = Dense(6, activation='softmax',kernel_regularizer=regularizers.l2(lambda_loss_amount))(outputs)
# keras.regularizers.l2(lambda_loss_amount)
adam = keras.optimizers.Adam(lr=learning_rate,epsilon=None, decay=0.0, amsgrad=False)

model = Model(inputs=sensor_input, outputs=outputs)
model.compile(loss='categorical_crossentropy',
      optimizer=adam,
      metrics=['accuracy'])

#model.fit([acc_inputs,gyro_inputs], train_y, batch_size=64, epochs=10)
model.summary()


W0917 17:09:45.217334 10600 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0917 17:09:45.242334 10600 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0917 17:09:45.244334 10600 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0917 17:09:45.281336 10600 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.comp

acc_conv3_shape [None, 20, 14, 64]


W0917 17:09:48.582455 10600 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 20, 120, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 18, 64)        1216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 18, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 20, 18, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 18, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 16, 64)        12352     
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 16, 64)        256       
__________

In [6]:
train_x = train_x.reshape((-1,20,120,1))
model.fit(train_x, train_y, batch_size=64,validation_split=0.2, epochs=50)

W0917 17:10:11.886041 10600 deprecation.py:323] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 95264 samples, validate on 23816 samples
Epoch 1/50
95264/95264 [==============================] - 624s 7ms/step - loss: 0.1704 - acc: 0.9414 - val_loss: 0.1369 - val_acc: 0.9654
Epoch 2/50
95264/95264 [==============================] - 608s 6ms/step - loss: 0.0507 - acc: 0.9850 - val_loss: 0.0923 - val_acc: 0.9792
Epoch 3/50
95264/95264 [==============================] - 610s 6ms/step - loss: 0.0354 - acc: 0.9900 - val_loss: 0.0852 - val_acc: 0.9811
Epoch 4/50
95264/95264 [==============================] - 610s 6ms/step - loss: 0.0273 - acc: 0.9928 - val_loss: 0.0762 - val_acc: 0.9814
Epoch 5/50
95264/95264 [==============================] - 609s 6ms/step - loss: 0.0231 - acc: 0.9945 - val_loss: 0.0755 - val_acc: 0.9821
Epoch 6/50
95264/95264 [==============================] - 611s 6ms/step - loss: 0.0194 - acc: 0.9957 - val_loss: 0.0577 - val_acc: 0.9877
Epoch 7/50
95264/95264 [==============================] - 610s 6ms/step - loss: 0.0181 - acc: 0.9960 - val_loss: 0.0655 - 

In [7]:
from keras.models import load_model
 
model.save('CNN_LSTM.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model
 
# returns a compiled model
# identical to the previous one
#model = load_model('DeepSense_keras.h5')

In [9]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn import metrics
test_x = eval_x.reshape((-1,20,120,1))
test_y = eval_y

# print ( "Loss = " + str( preds[0] ) )
# print ( "Test Accuracy = " + str( preds[1] ) )
print(model.metrics_names)
loss, acc = model.evaluate(test_x, test_y, batch_size=64)
print('loss, acc ',loss,acc)


y_pred = model.predict(test_x, batch_size=64, verbose=1)
# y_pred_bool = np.argmax(preds, axis=1)

for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0

print(classification_report(test_y, y_pred))
print(metrics.f1_score(test_y, y_pred, average="weighted"))
print(confusion_matrix(test_y.argmax(axis=1), y_pred.argmax(axis=1)))

['loss', 'acc']
1193/1193 [==============================] - 2s 2ms/step
loss, acc  0.5857870359432767 0.9002514659908771
1193/1193 [==============================] - 3s 2ms/step
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       118
           1       1.00      0.25      0.40         4
           2       0.96      1.00      0.98       231
           3       0.91      0.86      0.89       111
           4       0.87      0.86      0.86       360
           5       0.87      0.88      0.88       369

   micro avg       0.90      0.90      0.90      1193
   macro avg       0.93      0.80      0.83      1193
weighted avg       0.90      0.90      0.90      1193
 samples avg       0.90      0.90      0.90      1193

0.8994257682149092
[[111   0   7   0   0   0]
 [  1   1   2   0   0   0]
 [  1   0 230   0   0   0]
 [  0   0   0  96   6   9]
 [  2   0   0   7 310  41]
 [  0   0   0   2  41 326]]
